In [8]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

In [9]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 영화 리뷰를 작성하는 블로거입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="액션")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 영화 리뷰를 작성하는 블로거입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


### Multi Chain
**첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.**
**두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기**

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 리뷰를 작성하는 블로거입니다."),
    ("human", "{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")
])

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 리뷰를 작성하는 블로거입니다."),
    ("human", "{movie} 추전한 영화의 제목을 먼저 알려주시고, 감독, 줄거리, 등장인물 정보도 알려주세요.")
])

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [11]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

## **추천 영화: 존 윅(John Wick,2014)**

### **영화 정보**

* **장르:** 액션, 스릴러
* **감독:** 채드 스태헬스키
* **주연:** 키아누 리브스, 마이클 비 존스, 알피 앨런, 윌 포레이트

### **줄거리**

전직 킬러 존 윅(키아누 리브스)은 은퇴 후 평화로운 삶을 살고 있다. 하지만 그의 평화는 한때 존 윅의 차를 훔치고, 그의 개를 죽인 이오하임(알피 앨런)이라는 한 범죄 조직의 보스의 아들이 그의 집에 찾아오면서 깨진다. 이오하임은 존 윅에게 결투를 신청하고, 존 윅은 이오하임과 그의 조직원들을 모두 죽인다. 이 사건으로 인해 존 윅은 전 세계의 범죄 조직으로부터 표적이 된다.

### **리뷰**

존 윅은 화려한 액션과 스타일리시한 연출로 호평을 받은 영화입니다. 키아누 리브스의 뛰어난 액션 연기와 웅장한 스케일, 그리고 탄탄한 스토리가 이 영화를 돋보이게 합니다. 특히, 영화의 액션 장면들은 매우 인상적이며, 긴장감과 몰입감을 선사합니다. 또한, 영화의 세계관은 매우 독특하며, 범죄 조직의 규칙과 암살자들의 세계를 잘 표현하고 있습니다.

### **평가**

* **IMDB:**7.4/10
* **로튼 토마토:**86%

### **추천 이유**

* **액션:** 영화의 액션 장면들은 매우 화려하고 스타일리시합니다. 존 윅의 총격과 격투 장면들은 매우 인상적이며, 긴장감과 몰입감을 선사합니다.
* **연출:** 영화의 연출은 매우 독특하며, 범죄 조직의 규칙과 암살자들의 세계를 잘 표현하고 있습니다.
* **배우:** 키아누 리브스는 존 윅 역을 완벽하게 소화하며, 그의 연기는 매우 인상적입니다.
